In [2]:
import pandas as pd
import os
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, roc_curve, confusion_matrix




# Paths to predictions
TINYBERT_PATH = "../fl_tinybert/fl_tinybert/results"
XGB_PATH = "../results/xgboost"
clients = ["client_1", "client_2", "client_3", "client_4"]

# Directory for saving results
meta_results_dir = "../results/meta_fusion"
os.makedirs(meta_results_dir, exist_ok=True)

for test_client in clients:
    print(f"\n🔍 Running Leave-One-Client-Out Fusion for {test_client}...")

    # Create training data by excluding the test client
    train_clients = [client for client in clients if client != test_client]
    
    X_train, y_train = [], []
    
    for client in train_clients:
        tinybert_file = os.path.join(TINYBERT_PATH, client, "predictions.csv")
        xgb_file = os.path.join(XGB_PATH, f"{client}_xgb_predictions.csv")

        df_tiny = pd.read_csv(tinybert_file)
        df_xgb = pd.read_csv(xgb_file)

        # Extract features (probabilities)
        X_train.extend(np.vstack([
            df_tiny["Probability"].values,
            df_xgb["Probability"].values
        ]).T)
        
        y_train.extend(df_tiny["Actual"].values)

    # Convert to numpy arrays
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    # Train/test split
    tinybert_test_file = os.path.join(TINYBERT_PATH, test_client, "predictions.csv")
    xgb_test_file = os.path.join(XGB_PATH, f"{test_client}_xgb_predictions.csv")

    df_tiny_test = pd.read_csv(tinybert_test_file)
    df_xgb_test = pd.read_csv(xgb_test_file)

    X_test = np.vstack([
        df_tiny_test["Probability"].values,
        df_xgb_test["Probability"].values
    ]).T

    y_test = df_tiny_test["Actual"].values

    # Train the Meta-Fusion Classifier
    meta_model = LogisticRegression()
    meta_model.fit(X_train, y_train)

    # Make predictions
    y_pred = meta_model.predict(X_test)
    y_proba = meta_model.predict_proba(X_test)[:, 1]

    # Evaluation
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    print(f"✅ Accuracy: {acc:.4f}")
    print(f"✅ AUC: {auc:.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, digits=4))

    # Save confusion matrix plot
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Legit", "Phish"], yticklabels=["Legit", "Phish"])
    plt.title(f"{test_client} - Meta Fusion Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.tight_layout()
    cm_path = os.path.join(meta_results_dir, f"{test_client}_conf_matrix.png")
    plt.savefig(cm_path)
    plt.close()
    print(f"📊 Confusion matrix saved to {cm_path}")

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.figure()
    plt.plot(fpr, tpr, label=f"AUC = {auc:.4f}")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title(f"{test_client} - ROC Curve (Meta-Fusion)")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend()
    roc_path = os.path.join(meta_results_dir, f"{test_client}_roc_curve.png")
    plt.savefig(roc_path)
    plt.close()
    print(f"📈 ROC curve saved to {roc_path}")



🔍 Running Leave-One-Client-Out Fusion for client_1...
✅ Accuracy: 0.9980
✅ AUC: 0.9992

Classification Report:
              precision    recall  f1-score   support

           0     0.9980    0.9980    0.9980       500
           1     0.9980    0.9980    0.9980       500

    accuracy                         0.9980      1000
   macro avg     0.9980    0.9980    0.9980      1000
weighted avg     0.9980    0.9980    0.9980      1000

📊 Confusion matrix saved to ../results/meta_fusion/client_1_conf_matrix.png
📈 ROC curve saved to ../results/meta_fusion/client_1_roc_curve.png

🔍 Running Leave-One-Client-Out Fusion for client_2...
✅ Accuracy: 1.0000
✅ AUC: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000       313
           1     1.0000    1.0000    1.0000       313

    accuracy                         1.0000       626
   macro avg     1.0000    1.0000    1.0000       626
weighted avg     1.0000    1.0000 